In [32]:
import pandas as pd
import numpy as np
import math
import os
import neurokit2 as nk
# from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values, get_time_domain_features, get_frequency_domain_features, get_sampen
# from hrvanalysis import get_time_domain_features, get_frequency_domain_features, get_sampen
import nolds
from scipy import signal
import heartpy as hp

In [33]:
where_PPG = 'C:/Users/citioplab/Downloads/TVGH002'

# 前處理 acc

In [34]:
# 消除基線飄移
def filt_ma(acc):
    newlist = [x for x in acc if math.isnan(x) == False]
    b, a = signal.butter(3, 0.1, btype="highpass", output="ba")
    ma_acc = signal.filtfilt(b, a, newlist)
    return ma_acc

# main (先讀一位病人[02]一天的資料[4/2])

In [35]:
def time_imputation(df, column, filename):
    # 向下補值
    df[column].replace(0, np.nan, inplace=True)
    df[column].fillna(method='ffill', inplace=True)
    # 補開頭
    df[column].replace(np.nan, filename, inplace=True)
    # 統一格式
    df[column] = df[column].astype(str).str.rstrip('.0')
    df[column] = df[column].str.pad(width=14, side='right', fillchar='0')
    df[column] = pd.to_datetime(df[column])
    return df

In [36]:
# 讀取資料，並做成dataframe
no_list = ["02"]
new_df = pd.DataFrame(columns=['Measure_time', 'X_acc', 'Y_acc', 'Z_acc'])

for who in no_list:
    # goto = where_PPG + r'\TVGH0' + str(who) + "\\"
    goto = where_PPG + "\\"
    raw =  os.listdir(goto)
    for raw_data in raw:
        raw_data = "20190411164501_raw.csv"
        if "raw" in raw_data.split(".")[0]: 
            filename = raw_data.split(".")[0].replace('_raw.xlsx', '')[:14]
            df = pd.read_csv(goto + raw_data ,header = None)
            df = df.dropna()

            # 取出csv中的欄位值
            measure_time = np.array(df[0].values.tolist()).T[0]
            x_acc = filt_ma(np.array(df[[4]].values.tolist()).T[0]) # 消除基線飄移
            y_acc = filt_ma(np.array(df[[5]].values.tolist()).T[0]) # 消除基線飄移
            z_acc = filt_ma(np.array(df[[6]].values.tolist()).T[0]) # 消除基線飄移

            # 做成dataframe
            part_df = pd.DataFrame({'Measure_time': df[0].tolist(), 'X_acc':x_acc, 'Y_acc': y_acc, 'Z_acc': z_acc})
            file_start_time = filename[:13] + str((int(filename[13]) - 1))
            part_df = time_imputation(part_df, 'Measure_time', file_start_time)
            
            new_df = pd.concat([new_df, part_df], ignore_index=True)
        break


C:\Users\citioplab\AppData\Local\Temp\ipykernel_14268\669521990.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[column].fillna(method='ffill', inplace=True)
C:\Users\citioplab\AppData\Local\Temp\ipykernel_14268\2785910515.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, part_df], ignore_index=True)


In [37]:
period_counts_df = pd.DataFrame({'Measure_time': new_df['Measure_time'].value_counts().index, 'Count': new_df['Measure_time'].value_counts().values})
sorted_counts_df = period_counts_df.sort_values(by='Measure_time').reset_index(drop=True)
sorted_counts_df

,Measure_time,Count
0,2019-04-11 16:45:00,63
1,2019-04-11 16:45:01,64
2,2019-04-11 16:45:02,64
3,2019-04-11 16:45:03,64
4,2019-04-11 16:45:04,64
...,...,...
83445,2019-04-12 15:55:45,64
83446,2019-04-12 15:55:46,64
83447,2019-04-12 15:55:47,64
83448,2019-04-12 15:55:48,64


In [38]:
len(sorted_counts_df.loc[sorted_counts_df['Count'] == 127, 'Measure_time'])

0

# 計算SPT，得出sleep midpoint

## 0. 轉換成以秒為單位（原本是一秒鐘64筆資料） 

In [39]:
median_sec_X = pd.DataFrame(new_df.groupby('Measure_time')['X_acc'].mean())
median_sec_Y = pd.DataFrame(new_df.groupby('Measure_time')['Y_acc'].mean())
median_sec_Z = pd.DataFrame(new_df.groupby('Measure_time')['Z_acc'].mean())

In [40]:
df_sec = pd.DataFrame({'X_acc': median_sec_X['X_acc'], 'Y_acc': median_sec_Y['Y_acc'], 'Z_acc': median_sec_Z['Z_acc']})
df_sec

,X_acc,Y_acc,Z_acc
Measure_time,,,
2019-04-11 16:45:00,-3.084578,-12.233163,-3.571146
2019-04-11 16:45:01,3.627671,5.670447,1.086383
2019-04-11 16:45:02,-1.792854,-5.446903,-2.663697
2019-04-11 16:45:03,1.554613,4.036096,1.899621
2019-04-11 16:45:04,-0.530032,0.081657,3.203398
...,...,...,...
2019-04-12 15:55:45,0.665913,-5.784490,3.411770
2019-04-12 15:55:46,-1.529949,7.410318,-7.904562
2019-04-12 15:55:47,-1.717634,3.510017,1.692070


## 1. 5 second rolling medians of raw signals x, y, z

In [41]:
# Define the rolling window size (5 seconds)
window_size = 5

# Compute rolling medians for x, y, z coordinates
rolling_medians_x = df_sec['X_acc'].rolling(window=window_size).median()
rolling_medians_y = df_sec['Y_acc'].rolling(window=window_size).median()
rolling_medians_z = df_sec['Z_acc'].rolling(window=window_size).median()

# Combine rolling medians into a new DataFrame if needed
rolling_medians_df = pd.DataFrame({
    'rolling_median_x': rolling_medians_x,
    'rolling_median_y': rolling_medians_y,
    'rolling_median_z': rolling_medians_z
})

In [42]:
rolling_medians_df = rolling_medians_df.dropna()
rolling_medians_df

,rolling_median_x,rolling_median_y,rolling_median_z
Measure_time,,,
2019-04-11 16:45:04,-0.530032,0.081657,1.086383
2019-04-11 16:45:05,-0.466268,2.830867,1.086383
2019-04-11 16:45:06,-0.466268,2.830867,-2.616988
2019-04-11 16:45:07,-0.466268,2.830867,1.899621
2019-04-11 16:45:08,-0.466268,0.081657,-2.134911
...,...,...,...
2019-04-12 15:55:45,0.665913,-1.576766,4.745593
2019-04-12 15:55:46,-1.529949,-1.576766,3.411770
2019-04-12 15:55:47,-1.529949,1.427948,1.692070


## 2. calculate angle-z = (tan^-1(az/((ax^2+ay^2)^1/2)))*180/pi

In [43]:
def calculate_angle(acc_x, acc_y, acc_z):
    deg = math.atan(acc_x / (math.sqrt(acc_y**2 + acc_z**2)))*180/math.pi
    return deg

In [44]:
rolling_medians_df['angle'] = rolling_medians_df.apply(lambda row: calculate_angle(row['rolling_median_x'], row['rolling_median_y'], row['rolling_median_z']), axis=1)

In [45]:
rolling_medians_df

,rolling_median_x,rolling_median_y,rolling_median_z,angle
Measure_time,,,,
2019-04-11 16:45:04,-0.530032,0.081657,1.086383,-25.943611
2019-04-11 16:45:05,-0.466268,2.830867,1.086383,-8.742121
2019-04-11 16:45:06,-0.466268,2.830867,-2.616988,-6.896189
2019-04-11 16:45:07,-0.466268,2.830867,1.899621,-7.787981
2019-04-11 16:45:08,-0.466268,0.081657,-2.134911,-12.311312
...,...,...,...,...
2019-04-12 15:55:45,0.665913,-1.576766,4.745593,7.585127
2019-04-12 15:55:46,-1.529949,-1.576766,3.411770,-22.149423
2019-04-12 15:55:47,-1.529949,1.427948,1.692070,-34.644854


## 3. Succcesive 5 second average angle differences

In [46]:
# Compute rolling average for angles
rolling_avg_angle = pd.DataFrame(rolling_medians_df['angle'].rolling(window=5).mean())
rolling_avg_angle = rolling_avg_angle.dropna()
rolling_avg_angle

,angle
Measure_time,
2019-04-11 16:45:08,-12.336243
2019-04-11 16:45:09,-9.145581
2019-04-11 16:45:10,-6.892429
2019-04-11 16:45:11,-5.086534
2019-04-11 16:45:12,-2.958896
...,...
2019-04-12 15:55:45,7.795257
2019-04-12 15:55:46,-4.997248
2019-04-12 15:55:47,-4.273979


In [47]:
rolling_avg_angle['angle_diff'] = rolling_avg_angle['angle'].diff()
rolling_avg_angle = rolling_avg_angle.dropna()
rolling_avg_angle['angle_diff'] = abs(rolling_avg_angle['angle_diff'])
rolling_avg_angle

C:\Users\citioplab\AppData\Local\Temp\ipykernel_14268\1227082398.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rolling_avg_angle['angle_diff'] = abs(rolling_avg_angle['angle_diff'])


,angle,angle_diff
Measure_time,,
2019-04-11 16:45:09,-9.145581,3.190661
2019-04-11 16:45:10,-6.892429,2.253153
2019-04-11 16:45:11,-5.086534,1.805894
2019-04-11 16:45:12,-2.958896,2.127638
2019-04-11 16:45:13,0.073408,3.032304
...,...,...
2019-04-12 15:55:45,7.795257,6.490255
2019-04-12 15:55:46,-4.997248,12.792505
2019-04-12 15:55:47,-4.273979,0.723270


## 4. 5 minute rolling median of ang diff

In [48]:
rolling_avg_angle = rolling_avg_angle.reset_index()
rolling_avg_angle['time_diff'] = rolling_avg_angle['Measure_time'].diff()
rolling_avg_angle

,Measure_time,angle,angle_diff,time_diff
0,2019-04-11 16:45:09,-9.145581,3.190661,NaT
1,2019-04-11 16:45:10,-6.892429,2.253153,0 days 00:00:01
2,2019-04-11 16:45:11,-5.086534,1.805894,0 days 00:00:01
3,2019-04-11 16:45:12,-2.958896,2.127638,0 days 00:00:01
4,2019-04-11 16:45:13,0.073408,3.032304,0 days 00:00:01
...,...,...,...,...
83436,2019-04-12 15:55:45,7.795257,6.490255,0 days 00:00:01
83437,2019-04-12 15:55:46,-4.997248,12.792505,0 days 00:00:01
83438,2019-04-12 15:55:47,-4.273979,0.723270,0 days 00:00:01
83439,2019-04-12 15:55:48,-6.139600,1.865622,0 days 00:00:01


In [49]:
# 確認測量時間是連續的(無中斷)
rolling_avg_angle['time_diff'] = pd.to_timedelta(rolling_avg_angle['time_diff'])
filtered_df = rolling_avg_angle[rolling_avg_angle['time_diff'] != pd.Timedelta('0 days 00:00:01')]
filtered_df

,Measure_time,angle,angle_diff,time_diff
0,2019-04-11 16:45:09,-9.145581,3.190661,NaT


In [50]:
rolling_median_5min_angle = pd.DataFrame({
    'Measure_time': rolling_avg_angle['Measure_time'],
    'rolling_median_angle': rolling_avg_angle['angle_diff'].rolling(window=300).median()
})
rolling_median_5min_angle = rolling_median_5min_angle.dropna()
rolling_median_5min_angle

,Measure_time,rolling_median_angle
299,2019-04-11 16:50:08,8.416869
300,2019-04-11 16:50:09,8.416869
301,2019-04-11 16:50:10,8.416869
302,2019-04-11 16:50:11,8.441510
303,2019-04-11 16:50:12,8.441510
...,...,...
83436,2019-04-12 15:55:45,7.986642
83437,2019-04-12 15:55:46,7.986642
83438,2019-04-12 15:55:47,7.915763
83439,2019-04-12 15:55:48,7.854811


## 5. detect values < 10th percentile of angle diff

In [51]:
rolling_median_5min_angle.describe()

,Measure_time,rolling_median_angle
count,83142,83142.000000
mean,2019-04-12 04:22:58.499999744,6.669689
min,2019-04-11 16:50:08,3.052112
25%,2019-04-11 22:36:33.249999872,5.680347
50%,2019-04-12 04:22:58.500000,6.699315
75%,2019-04-12 10:09:23.750000128,7.581159
max,2019-04-12 15:55:49,10.909523
std,NaN,1.309521


In [52]:
percentile_90 = rolling_median_5min_angle['rolling_median_angle'].quantile(0.9)
percentile_90

8.342959893171766

In [53]:
rolling_median_5min_angle['still'] = rolling_median_5min_angle['rolling_median_angle'].apply(lambda x: 1 if x < percentile_90 else 0)
rolling_median_5min_angle

,Measure_time,rolling_median_angle,still
299,2019-04-11 16:50:08,8.416869,0
300,2019-04-11 16:50:09,8.416869,0
301,2019-04-11 16:50:10,8.416869,0
302,2019-04-11 16:50:11,8.441510,0
303,2019-04-11 16:50:12,8.441510,0
...,...,...,...
83436,2019-04-12 15:55:45,7.986642,1
83437,2019-04-12 15:55:46,7.986642,1
83438,2019-04-12 15:55:47,7.915763,1
83439,2019-04-12 15:55:48,7.854811,1


In [54]:
rolling_median_5min_angle['still'].value_counts()

still
1    74819
0     8323
Name: count, dtype: int64

## 6. keep blocks that last > 30 min

In [55]:
rolling_median_5min_angle.head(20)

,Measure_time,rolling_median_angle,still
299,2019-04-11 16:50:08,8.416869,0
300,2019-04-11 16:50:09,8.416869,0
301,2019-04-11 16:50:10,8.416869,0
302,2019-04-11 16:50:11,8.441510,0
303,2019-04-11 16:50:12,8.441510,0
304,2019-04-11 16:50:13,8.447022,0
305,2019-04-11 16:50:14,8.447022,0
306,2019-04-11 16:50:15,8.529649,0
307,2019-04-11 16:50:16,8.529649,0
308,2019-04-11 16:50:17,8.641416,0


In [56]:
# 找到所有的blocks
block_start = []
block_end = []
pre_value = 0

for idx, value in enumerate(rolling_median_5min_angle['still']):
    if pre_value == 0 and value == 1:
        block_start.append(rolling_median_5min_angle['Measure_time'].iloc[idx])
    elif pre_value == 1 and value == 0:
        block_end.append(rolling_median_5min_angle['Measure_time'].iloc[idx-1])
    pre_value = value

In [57]:
block_duration = [block_end[i] - block_start[i] for i in range(len(block_end))]
print(block_duration)

[Timedelta('0 days 01:04:45'), Timedelta('0 days 00:00:00'), Timedelta('0 days 00:16:45'), Timedelta('0 days 00:01:30'), Timedelta('0 days 00:54:46'), Timedelta('0 days 00:00:00'), Timedelta('0 days 00:00:08'), Timedelta('0 days 01:01:03'), Timedelta('0 days 00:00:00'), Timedelta('0 days 00:00:02'), Timedelta('0 days 00:43:27'), Timedelta('0 days 00:00:13'), Timedelta('0 days 00:00:08'), Timedelta('0 days 00:00:33'), Timedelta('0 days 00:04:31'), Timedelta('0 days 00:00:38'), Timedelta('0 days 00:41:21'), Timedelta('0 days 00:00:29'), Timedelta('0 days 00:01:15'), Timedelta('0 days 00:00:14'), Timedelta('0 days 00:00:12'), Timedelta('0 days 00:00:07'), Timedelta('0 days 00:00:53'), Timedelta('0 days 00:00:03'), Timedelta('0 days 00:00:00'), Timedelta('0 days 00:00:00'), Timedelta('0 days 00:00:00'), Timedelta('0 days 00:00:02'), Timedelta('0 days 00:00:34'), Timedelta('0 days 00:07:49'), Timedelta('0 days 00:00:13'), Timedelta('0 days 00:14:58'), Timedelta('0 days 00:00:09'), Timedelta

In [ ]:
blocks_df = pd.DataFrame({'block_start': block_start[:-1], 'block_end': block_end, 'duration':block_duration})
blocks_df

In [59]:
blocks_df[blocks_df['duration'] > pd.Timedelta('0 days 00:30:00')]

,block_start,block_end,duration
0,2019-04-11 16:52:25,2019-04-11 17:57:10,0 days 01:04:45
4,2019-04-11 18:21:31,2019-04-11 19:16:17,0 days 00:54:46
7,2019-04-11 19:18:29,2019-04-11 20:19:32,0 days 01:01:03
10,2019-04-11 20:19:45,2019-04-11 21:03:12,0 days 00:43:27
16,2019-04-11 21:11:36,2019-04-11 21:52:57,0 days 00:41:21
45,2019-04-11 23:14:41,2019-04-12 00:29:04,0 days 01:14:23
46,2019-04-12 00:29:41,2019-04-12 02:42:44,0 days 02:13:03
48,2019-04-12 02:43:34,2019-04-12 03:59:24,0 days 01:15:50
54,2019-04-12 04:18:50,2019-04-12 05:09:33,0 days 00:50:43
55,2019-04-12 05:13:30,2019-04-12 05:59:59,0 days 00:46:29


## 7. combine blocks with time gap < 60 min

In [60]:
# filter出blocks
spt_blocks = blocks_df[blocks_df['duration'] > pd.Timedelta('0 days 00:30:00')]
# for col in ['block_start', 'block_end']:
#     spt_blocks[col] = pd.to_datetime(spt_blocks[col]).dt.time
spt_blocks

,block_start,block_end,duration
0,2019-04-11 16:52:25,2019-04-11 17:57:10,0 days 01:04:45
4,2019-04-11 18:21:31,2019-04-11 19:16:17,0 days 00:54:46
7,2019-04-11 19:18:29,2019-04-11 20:19:32,0 days 01:01:03
10,2019-04-11 20:19:45,2019-04-11 21:03:12,0 days 00:43:27
16,2019-04-11 21:11:36,2019-04-11 21:52:57,0 days 00:41:21
45,2019-04-11 23:14:41,2019-04-12 00:29:04,0 days 01:14:23
46,2019-04-12 00:29:41,2019-04-12 02:42:44,0 days 02:13:03
48,2019-04-12 02:43:34,2019-04-12 03:59:24,0 days 01:15:50
54,2019-04-12 04:18:50,2019-04-12 05:09:33,0 days 00:50:43
55,2019-04-12 05:13:30,2019-04-12 05:59:59,0 days 00:46:29


In [61]:
# 計算time gap
time_gap = []
for idx, value in enumerate(spt_blocks['block_end']):
    if idx < len(spt_blocks)-1:
        time_gap.append(spt_blocks['block_start'].iloc[idx+1] - spt_blocks['block_end'].iloc[idx])
    else:
        time_gap.append(np.nan)
spt_blocks['time_gap'] = time_gap
spt_blocks

C:\Users\citioplab\AppData\Local\Temp\ipykernel_14268\1652229148.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spt_blocks['time_gap'] = time_gap


,block_start,block_end,duration,time_gap
0,2019-04-11 16:52:25,2019-04-11 17:57:10,0 days 01:04:45,0 days 00:24:21
4,2019-04-11 18:21:31,2019-04-11 19:16:17,0 days 00:54:46,0 days 00:02:12
7,2019-04-11 19:18:29,2019-04-11 20:19:32,0 days 01:01:03,0 days 00:00:13
10,2019-04-11 20:19:45,2019-04-11 21:03:12,0 days 00:43:27,0 days 00:08:24
16,2019-04-11 21:11:36,2019-04-11 21:52:57,0 days 00:41:21,0 days 01:21:44
45,2019-04-11 23:14:41,2019-04-12 00:29:04,0 days 01:14:23,0 days 00:00:37
46,2019-04-12 00:29:41,2019-04-12 02:42:44,0 days 02:13:03,0 days 00:00:50
48,2019-04-12 02:43:34,2019-04-12 03:59:24,0 days 01:15:50,0 days 00:19:26
54,2019-04-12 04:18:50,2019-04-12 05:09:33,0 days 00:50:43,0 days 00:03:57
55,2019-04-12 05:13:30,2019-04-12 05:59:59,0 days 00:46:29,0 days 02:24:22


## 8. Find the longest time block = SPT window

In [62]:
SPT_start = 0
SPT_end = 0
SPT_duration = pd.Timedelta('0 days 00:00:00')
temp_start = 0
temp_end = 0
temp_duration = pd.Timedelta('0 days 00:00:00')

for idx, value in enumerate(spt_blocks['time_gap']):
    if spt_blocks['time_gap'].iloc[idx] < pd.Timedelta('0 days 00:60:00'):
        if idx == 0:
            temp_start = spt_blocks['block_start'].iloc[idx]
        elif spt_blocks['time_gap'].iloc[idx-1] >  pd.Timedelta('0 days 00:60:00'):
            temp_start = spt_blocks['block_start'].iloc[idx]

        temp_end = spt_blocks['block_end'].iloc[idx+1]
        temp_duration += spt_blocks['duration'].iloc[idx]
        
    else:
        if temp_duration > SPT_duration:
            SPT_duration = temp_duration
            SPT_start = temp_start
            SPT_end = temp_end
        temp_duration = pd.Timedelta('0 days 00:00:00')


## 9. Mark the start and end time of the SPT window

In [63]:
SPT_start

Timestamp('2019-04-11 23:14:41')

In [64]:
SPT_end

Timestamp('2019-04-12 05:59:59')

## 10. Calculate the SPT window midpoints and their standard deviations

In [65]:
SPT_midpoint = SPT_start + SPT_duration/2
SPT_midpoint

Timestamp('2019-04-12 02:01:40.500000')